In [1]:
%run imports
from src.functions import *
from src.classes import *
from src.plots import *
#195633320791780608, 195633325090480896


In [3]:
mystar = Star(195633325090480896)
mystar.SkyCoord

<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, pc)
    [(81.6256439, 42.93974983, 3138.43115)]
 (pm_ra_cosdec, pm_dec) in mas / yr
    [(1.408, -2.592)]>

In [2]:
coord = SkyCoord.from_name("SNR G166.0+04.3")
radius = 1*u.deg
# stars_in_region = get_stars_in_region(coord, radius)
stars_in_region = Table.read('data/stars_near_g166.ecsv', format='ascii.ecsv')
# stars_in_region
dist_mask_u = stars_in_region['rgeo']<4.7*u.kpc
dist_mask_l = stars_in_region['rgeo']>3*u.kpc
ruwe_mask = stars_in_region['RUWE']<1.4
plxq_mask = stars_in_region['Plx']/stars_in_region['e_Plx'] > 5
sir = stars_in_region[dist_mask_l & dist_mask_u & ruwe_mask & plxq_mask]
# sir

In [4]:
matches_file = "/home/suro/suro_aiu/traceback/main_g166/matches/1pc.003+004"
best_params(matches_file)

TypeInput,RA,DE,Plx,e_Plx,RV,e_RV,RVdist,pmRA,e_pmRA,pmDE,e_pmDE,Source
int64,str12,str12,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
1,05 26 25.573,42 57 40.320,0.36088,0.0,0.0,0.0,0.0,0.75017,0.0,-3.93651,0.0,0.0
1,05 26 25.846,42 57 38.820,0.36088,0.0,0.0,0.0,0.0,1.42342,0.0,-2.57198,0.0,0.0


In [54]:
traj_file = "/home/suro/suro_aiu/traceback/main_g166/trajectories/best_match_traj"
table = read_trajectory_out(traj_file)
table

time,l_1,b_1,dist_1,l_2,b_2,dist_2,sep3d
yr,deg,deg,pc,deg,deg,pc,pc
int64,float64,float64,float64,float64,float64,float64,float64
0,166.11239,4.27749,2771.00421,166.11101,4.28173,2771.00421,0.2157
-100,166.11213,4.27753,2771.00421,166.11077,4.28174,2771.00421,0.2137
-200,166.11186,4.27757,2771.00421,166.11052,4.28174,2771.00421,0.2117
-300,166.1116,4.27762,2771.00421,166.11028,4.28175,2771.00421,0.20971
-400,166.11134,4.27766,2771.00421,166.11004,4.28176,2771.00421,0.20771
-500,166.11108,4.2777,2771.00421,166.1098,4.28176,2771.00421,0.20571
-600,166.11081,4.27775,2771.00421,166.10956,4.28177,2771.00421,0.20372
-700,166.11055,4.27779,2771.00421,166.10932,4.28178,2771.00421,0.20173


In [53]:
from astropy.coordinates import Galactocentric
table['SkyCoord1_galactic'] = SkyCoord(l=table['l_1'],
                             b=table['b_1'],
                             distance=table['dist_1'],
                             frame='galactic'
                             )
table['SkyCoord2_galactic'] = SkyCoord(l=table['l_2'],
                             b=table['b_2'],
                             distance=table['dist_2'],
                             frame='galactic'
                             )
table['SkyCoord1_galactocentric'] = table['SkyCoord1_galactic'].transform_to(Galactocentric)
table['SkyCoord2_galactocentric'] = table['SkyCoord2_galactic'].transform_to(Galactocentric)


In [52]:
import plotly.graph_objects as go

# Step 1: Create the scatter plot for the first line with both markers and lines
trace1 = go.Scatter3d(
    x=table['SkyCoord1_galactocentric'].x,  # x-axis (l_1)
    y=table['SkyCoord1_galactocentric'].y,  # y-axis (b_1)
    z=table['SkyCoord1_galactocentric'].z,  # z-axis (dist_1)
    mode='lines+markers',  # Show both markers and lines
    name='Star1',  # Legend name
    marker=dict(size=1),  # Size of the scatter points
    line=dict(width=1),  # Width of the line
    hovertext=table['time'],
    hoverinfo="text"
)

# Step 2: Create the scatter plot for the second line with both markers and lines
trace2 = go.Scatter3d(
    x=table['SkyCoord2_galactocentric'].x,  # x-axis (l_2)
    y=table['SkyCoord2_galactocentric'].y,  # y-axis (b_2)
    z=table['SkyCoord2_galactocentric'].z,  # z-axis (dist_2)
    mode='lines+markers',  # Show both markers and lines
    name='Star2',  # Legend name
    marker=dict(size=1),  # Size of the scatter points
    line=dict(width=1),  # Width of the line
    hovertext=table['time'],
    hoverinfo="text"
)

# Step 3: Combine the traces and create the layout
data = [trace1, trace2]  # Combine both lines

layout = go.Layout(
    title='3D Scatter Plot with Lines',  # Title of the plot
    scene=dict(
        xaxis_title='X (pc)',
        yaxis_title='Y (pc)',
        zaxis_title='Z (pc)',
    )
)

# Step 4: Create the figure and plot it
fig = go.Figure(data=data, layout=layout)
fig.show()
